In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [2]:
df = pd.read_csv('memphis_restaurants.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [32]:
df_small = df[0:5]

In [38]:
# for every company
for row in df_small.iterrows():
    try:
        #### Get the business name and url ####
        business_name = row[1]['name']
        business_url = row[1]['url']
        print(business_name, business_url)

        #### Request the page ####
        html = requests.get(business_url)
        soup = BeautifulSoup(html.content, 'html.parser')
        
        #### Get the type of restaurant
        restaurant_type = soup.select('.css-bq71j2 .css-166la90')
        types = ''
        for i in restaurant_type:
            types += (i.string+',')
        
        
        #### Get the adress
        adress = soup.select('.css-1h1j0y3 .css-e81eai')
        for i in adress:
            adress = i.string
            
            
        #### Get the health score
        # check whether they have one and get the score
        try:
            health_score = soup.select('.label-spacing-v2__373c0__PBYkt')
            for i in health_score:
                health_score = int(re.sub(' out of 100', '', i.string))
        # if they have no, put it as a missing value
        except:
            health_score = ''
        
        
        #### Get the number of pages for reviews and limit it to 10 ####
        n_reviews = soup.select('.text-align--center__373c0__1l506 .css-e81eai')
        try:
            n_review_pg = int(re.sub('1 of ', '',n_reviews[0].string))
            if n_review_pg > 10:
                n_review_pg = 10
        except:
            n_review_pg = 10

            
        #### Get the data for each of the review pages 2 to 10 ####
        # for every review
        for i in range(n_review_pg):
            try:
                url2 = business_url + '?start=' + str(i * 10)
                html = requests.get(url2)
                soup = BeautifulSoup(html.content, 'html.parser')

                #### Names
                names = soup.select('.css-m6anxm .css-166la90')
                review_names = []

                for name in names:
                    review_names.append(name.string)

                #### Ratings
                ratings = soup.select('.margin-b1-5__373c0__2Wblx .overflow--hidden__373c0__2B0kz')

                review_ratings = []

                for rating in ratings:
                    review_ratings.append(float(re.sub(' star rating',
                                                       '',rating.attrs['aria-label'])))
                #### Dates
                dates = soup.select('.margin-b1-5__373c0__2Wblx .css-e81eai')

                review_dates = []

                for date in dates:
                    review_dates.append(date.string)

                #### Text
                texts = soup.select('.css-n6i4z7 .raw__373c0__3rcx7')
                review_texts = []

                for text in texts:
                    review_texts.append(text.get_text())

                name_business_mult = [business_name] * len(review_names)
                url_business_mult = [business_url] * len(review_names)
                restaurant_type_mult = [restaurant_type] * len(review_names)
                health_score_mult = [health_score] * len(review_names)
                adress_mult = [adress] * len(review_names)
                
                #### Check whether the review has a picture included or not
                picture = soup.select('')
                picture_available = []
                
                # Adding all parts together
                review_data = list(zip(name_business_mult, url_business_mult,
                                   review_names, review_ratings, review_dates,
                                   review_texts, adress_mult, health_score_mult, restaurant_type_mult))

                df_reviews = pd.DataFrame(data=review_data,
                                          columns = ['name_business', 'url',
                                                     'username', 'rating',
                                                     'date_review', 'text', 'adress', 'health_score',
                                                      'restaurant_type'])

                with open('memphis_reviews.csv', 'a', newline='') as f:
                        df_reviews.to_csv(f, index=False, header=False)

                print('Review page ' + str(i))
                time.sleep(4)

            except Exception as e:
                print('Error in second stage')
                print(e)
                
        print(business_name)
        time.sleep(5)

    except Exception as e:
        print('Error in first stage')
        print(e)

The Second Line https://www.yelp.com/biz/the-second-line-memphis
Error in second stage
Expected a selector at position 0
  line 1:

^
Error in second stage
Expected a selector at position 0
  line 1:

^
Error in second stage
Expected a selector at position 0
  line 1:

^
Error in second stage
Expected a selector at position 0
  line 1:

^
Error in second stage
Expected a selector at position 0
  line 1:

^
The Second Line
Bounty On Broad https://www.yelp.com/biz/bounty-on-broad-memphis
Error in second stage
Expected a selector at position 0
  line 1:

^
Error in second stage
Expected a selector at position 0
  line 1:

^
Error in second stage
Expected a selector at position 0
  line 1:

^
Error in second stage
Expected a selector at position 0
  line 1:

^
Error in second stage
Expected a selector at position 0
  line 1:

^
Bounty On Broad
Curfew https://www.yelp.com/biz/curfew-memphis
Error in second stage
Expected a selector at position 0
  line 1:

^
Error in second stage
Expected a

In [ ]:
test = pd.read_csv('memphis_reviews.csv', header=None, encoding='ISO-8859-1')

In [ ]:
test

